# License Plate Recognition - One Click Test
Chỉ cần thay tên file ảnh và chạy một cell duy nhất để nhận diện biển số xe và hiển thị toàn bộ các bước xử lý.

In [ ]:
# Import Required Libraries
from PIL import ImageFont, ImageDraw, Image
import numpy as np
import cv2
from easyocr import Reader
import matplotlib.pyplot as plt

In [ ]:
def plate_recognition(image_path):
    # Load and resize image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (800, 600))
    
    # Load font
    fontpath = "./arial.ttf"
    font = ImageFont.truetype(fontpath, 32)
    text_color = (0, 255, 0, 0)
    
    # Display original image
    plt.figure(figsize=(8,6))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title('1. Ảnh gốc')
    plt.axis('off')
    plt.show()
    
    # Convert to grayscale
    grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    plt.figure(figsize=(8,6))
    plt.imshow(grayscale, cmap='gray')
    plt.title('2. Ảnh xám')
    plt.axis('off')
    plt.show()
    
    # Gaussian Blur
    blurred = cv2.GaussianBlur(grayscale, (5, 5), 0)
    plt.figure(figsize=(8,6))
    plt.imshow(blurred, cmap='gray')
    plt.title('3. Làm mờ Gaussian')
    plt.axis('off')
    plt.show()
    
    # Canny Edge Detection
    edged = cv2.Canny(blurred, 10, 200)
    plt.figure(figsize=(8,6))
    plt.imshow(edged, cmap='gray')
    plt.title('4. Phát hiện biên (Canny)')
    plt.axis('off')
    plt.show()
    
    # Find and draw contours
    contours, _ = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
    contour_img = img.copy()
    cv2.drawContours(contour_img, contours, -1, (0, 255, 0), 2)
    plt.figure(figsize=(8,6))
    plt.imshow(cv2.cvtColor(contour_img, cv2.COLOR_BGR2RGB))
    plt.title('5. Các contour lớn nhất')
    plt.axis('off')
    plt.show()
    
    # Detect license plate region
    number_plate_shape = None
    for c in contours:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if len(approx) == 4:
            number_plate_shape = approx
            break
    if number_plate_shape is not None:
        plate_img = img.copy()
        cv2.drawContours(plate_img, [number_plate_shape], -1, (255, 0, 0), 3)
        plt.figure(figsize=(8,6))
        plt.imshow(cv2.cvtColor(plate_img, cv2.COLOR_BGR2RGB))
        plt.title('6. Khung biển số xe')
        plt.axis('off')
        plt.show()
    else:
        print('Không tìm thấy hình dạng biển số xe!')
    
    # Extract license plate ROI
    if number_plate_shape is not None:
        (x, y, w, h) = cv2.boundingRect(number_plate_shape)
        number_plate = grayscale[y:y+h, x:x+w]
        plt.figure(figsize=(8,6))
        plt.imshow(number_plate, cmap='gray')
        plt.title('7. Ảnh cắt biển số')
        plt.axis('off')
        plt.show()
    else:
        number_plate = None
        print('Không thể cắt ROI biển số vì không tìm thấy khung!')
    
    # OCR
    if number_plate is not None:
        reader = Reader(['en'])
        detection = reader.readtext(number_plate)
    else:
        detection = []
    
    # Display final result
    img_result = img.copy()
    if number_plate_shape is not None:
        cv2.drawContours(img_result, [number_plate_shape], -1, (255, 0, 0), 3)
    if len(detection) == 0:
        text = "Không thấy bảng số xe"
    else:
        text = "Biển số: " + detection[0][1]
    img_pil = Image.fromarray(img_result)
    draw = ImageDraw.Draw(img_pil)
    draw.text((200, 500), text, font=font, fill=text_color)
    img_result = np.array(img_pil)
    plt.figure(figsize=(8,6))
    plt.imshow(cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB))
    plt.title('8. Kết quả dự đoán + khung biển số')
    plt.axis('off')
    plt.show()

In [ ]:
# Chạy nhận diện biển số với ảnh mong muốn
plate_recognition('image3.jpg')  # Thay bằng tên file ảnh khác để test